In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
import sys
import os

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

Notes:
* Going to take the top 75 scoring players in the leage
    - in 1995-96 season, there were only 92 players in the leage
    - lowest ranked scorer that won the mvp was Steve Nash in 2005 where he averaged 15.5 PPG (63rd in the leage)
* csv files are in the directory of the 2nd year in the season. ex. the 95-96 season will be in the 1996 directory
* in the directory for the year, there is a players.csv file and a teams.csv file
    - the teams.csv file is so that I can associate a team record with the players
* the mvp.csv dataset is in the outer data dir and has the name of each mvp winner since the 65-66 season


**GET DATA SAMPLE**

In [2]:
# get the data
def get_data(year):
    players = pd.read_csv("data/" + year + "/players.csv")
    return players

In [3]:
# transform the player data into something uniform that we can use
def get_top_75_filtered(players):
    
    # remove entries for players that played on multiple teams, just look at the last team they played for
    players_filtered = players.drop_duplicates(subset='Player', keep='last').reset_index()
    
    # sort by points per game and take top 75 candidates
    players_sorted = players_filtered.sort_values(by="PTS", ascending=False)
    top_75 = players_sorted.head(75).copy()
    
    # metrics we are using
    metrics = ["Player", "Rk", "Age", "Team", "Pos", "G", "GS", "MP", "FG", "FGA", "FG%", "3P", "3P%", "2P", "2PA", "2P%", "eFG%", "FT", "FTA", "FT%", "ORB", "DRB", "TRB", "AST", "STL", "BLK", "TOV", "PF", "PTS"]

    # these attributes didn't exist in 1956, so we have to create them
    # ['GS', '3P', '3P%', '2P', '2PA', '2P%', 'eFG%', 'ORB', 'DRB', 'STL', 'BLK', 'TOV']
    
    # assume they started every game they played
    if 'GS' not in top_75.columns:
        top_75['GS'] = top_75['G']
    
    # if data came before 3 pointers were invented, they only shot 2 pointers
    if '3P' not in top_75.columns:
        top_75['3P'] = 0
        top_75['3P%'] = 0
        top_75['2P'] = top_75['FG']
        top_75['2PA'] = top_75['FGA']
        top_75['2P%'] = top_75['FG%']
        top_75['eFG%'] = top_75['FG%']
    
    # if they didn't track offensive/deffensive rebounds, assume 3/4 of the rebounds were on defense
    if 'ORB' not in top_75.columns or 'DRB' not in top_75.columns:
        top_75['ORB'] = top_75['TRB']/4
        top_75['DRB'] = top_75['TRB'] - top_75['ORB']
    
    # assume steals, blocks, and turnovers are all 0 unless specified
    if 'STL' not in top_75.columns:
        top_75['STL'] = 0
    
    if 'BLK' not in top_75.columns:
        top_75['BLK'] = 0
    
    if 'TOV' not in top_75.columns:
        top_75['TOV'] = 0
    
    top_75_filtered = top_75[metrics]
    return top_75_filtered


In [4]:
# get the team data and turn it into something usable

def get_team_data(year):
    teams_vs = pd.read_csv("data/" + year + "/teams.csv")
    teams_vs = teams_vs.drop(columns=['Rk'])
    
    teams = pd.DataFrame(columns=['Team', 'Abbreviation', 'Wins', 'Losses', 'Win%'])
    
    # Get abbreviation and record
    for index, row in teams_vs.iterrows():
    
        # extract wins and losses
        row_wins = 0
        row_losses = 0
        for record in row:
            if pd.notna(record) and len(record) < 6:
                win_loss = record.split('-')
                row_wins += int(win_loss[0])
                row_losses += int(win_loss[1])
    
        # print("The " + row['Team'] + " have won ", row_wins, " and lost ", row_losses, " games")
    
        # don't divide by 0
        if row_losses == 0:
            win_pct = 1
        else:
            win_pct = row_wins/(row_losses+row_wins)
    
        # add to df    
        teams.loc[len(teams)] = ({'Team': row['Team'], 'Abbreviation': row.index[row.name + 1], 'Wins': row_wins, 'Losses': row_losses, 'Win%': win_pct})
    
    # get overall team rank in the nba
    teams = teams.sort_values(by='Win%', ascending=False)
    teams = teams.reset_index(drop=True)
    teams['Rank'] = teams.index+1
    
    return teams

# get_team_data("2025")

In [5]:
# combine into final dataframe
def merge_data(top_75_filtered, teams):
    player_season_final_df = pd.merge(top_75_filtered, teams, left_on='Team', right_on='Abbreviation')
    # print(player_season_final_df.head())
    return player_season_final_df

In [6]:
# verify that all teams matched, this should print an empty dataframe
def verify_team_match(top_75_filtered, teams):
    unmatched_teams = top_75_filtered[~top_75_filtered['Team'].isin(teams['Abbreviation'])]
    if len(unmatched_teams['Team']) > 0:
        print(unmatched_teams)
        return 1
    return 0
        

In [7]:
# only keep numerical variables, don't worry about player name, team name, etc
def get_numerical_df(player_season_final_df):
    numerical_df = player_season_final_df.select_dtypes(include=['number'])
    return numerical_df

In [8]:
def get_mvp_by_year(year):
    mvps = pd.read_csv("data/mvps.csv")
    result = mvps.loc[mvps['Season'].str[:4].astype(int) == int(year) - 1, 'Player'].values

    if(len(result) == 1):
        return result[0]
    else:
        print("found ", len(result), " MVPs in year " + year)

In [9]:
def get_final_df_by_year(year):
    # get the data
    og_data = get_data(year)
    top_75 = get_top_75_filtered(og_data)
    team_data = get_team_data(year)

    # merge it
    player_team_combined = merge_data(top_75, team_data)

    # add year and MVP attribute
    player_team_combined['Year'] = int(year)

    # mvp = get_mvp_by
    player_team_combined['MVP'] = (player_team_combined['Player'] == get_mvp_by_year(year)).astype(int)    

    # verify merge worked correctly
    if verify_team_match(top_75, team_data) > 0:
        print("Team merge did not work correctly. Aborting")
        sys.exit()

    # print(player_team_combined.head())

    # get only numeric variables
    numerical_df = get_numerical_df(player_team_combined)
    return numerical_df

# get_final_df_by_year("2025")

In [10]:
def get_all_years_arr():
    scaler = StandardScaler()
    
    all_years_df = []
    # get all the data from all the years
    for year_dir in os.listdir("data"):

        #skip current year
        if year_dir == "2025": # or (year_dir != "mvps.csv" and int(year_dir) < 1980):
            continue
            
        # verify that the item is a directory and not a file. the mvps.csv file will be ignored
        if os.path.isdir(os.path.join("data", year_dir)):
            year_df = get_final_df_by_year(year_dir)
            all_years_df.append(year_df)

    return np.array(all_years_df)
       
    


**MAKE MODEL**

In [11]:
tf.random.set_seed(42)
model = tf.keras.Sequential()

model.add(tf.keras.layers.InputLayer(input_shape=[75, 31]))

model.add(tf.keras.layers.LayerNormalization(axis=-1))

# Apply TimeDistributed Dense layers to process each player's features independently
model.add(tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(300, activation="relu")))
model.add(tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(300, activation="relu")))

# Output layer: Process each player independently, without activation
model.add(tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(1)))

model.add(tf.keras.layers.Flatten())

# Apply softmax across the 75 players
model.add(tf.keras.layers.Dense(75, activation="softmax"))


C:\Python311\Lib\site-packages\keras\src\layers\core\input_layer.py:26: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


In [12]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ layer_normalization                  │ (None, 75, 31)              │              62 │
│ (LayerNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ time_distributed (TimeDistributed)   │ (None, 75, 300)             │           9,600 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ time_distributed_1 (TimeDistributed) │ (None, 75, 300)             │          90,300 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ time_distributed_2 (TimeDistributed) │ (None, 75, 1)               │             301 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 75)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 75)                  │           5,700 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 105,963 (413.92 KB)

 Trainable params: 105,963 (413.92 KB)

 Non-trainable params: 0 (0.00 B)

In [13]:
# optimizer = tf.keras.optimizers.Adam(learning_rate=1e-2)
optimizer = tf.keras.optimizers.SGD(learning_rate=0.01)
model.compile(loss="categorical_crossentropy",
              optimizer=optimizer,
              metrics=["accuracy"])

**TRAIN MODEL**

In [14]:
# get data for model
all_years_arr = get_all_years_arr()

#split into training/validation/test
train_arr, temp_arr = train_test_split(all_years_arr, test_size=0.3, random_state=42)
val_arr, test_arr = train_test_split(temp_arr, test_size=0.5, random_state=42)

# Separate features (X) and labels (y)
X_train = train_arr[:, :, :-1]
y_train = train_arr[:, :, -1]

X_val = val_arr[:, :, :-1]
y_val = val_arr[:, :, -1]

X_test = test_arr[:, :, :-1]
y_test = test_arr[:, :, -1]

In [15]:
history = model.fit(X_train, y_train, epochs=30, batch_size=1, validation_data=(X_val, y_val))

Epoch 1/30
48/48 ━━━━━━━━━━━━━━━━━━━━ 29s 129ms/step - accuracy: 0.0820 - loss: 4.2268 - val_accuracy: 0.2000 - val_loss: 3.9221
Epoch 2/30
48/48 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - accuracy: 0.1811 - loss: 3.7158 - val_accuracy: 0.2000 - val_loss: 3.7315
Epoch 3/30
48/48 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - accuracy: 0.1640 - loss: 3.2231 - val_accuracy: 0.2000 - val_loss: 3.6867
Epoch 4/30
48/48 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - accuracy: 0.1218 - loss: 2.9144 - val_accuracy: 0.2000 - val_loss: 3.6741
Epoch 5/30
48/48 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - accuracy: 0.1218 - loss: 2.7911 - val_accuracy: 0.2000 - val_loss: 3.6651
Epoch 6/30
48/48 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - accuracy: 0.1218 - loss: 2.7437 - val_accuracy: 0.2000 - val_loss: 3.6656
Epoch 7/30
48/48 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - accuracy: 0.1218 - loss: 2.7208 - val_accuracy: 0.2000 - val_loss: 3.6740
Epoch 8/30
48/48 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - accuracy: 0.1218 - loss: 2.7078 - val_accuracy: 0.2000 -

In [16]:
input_data = np.random.random((1, 75, 31))
predictions = model.predict(input_data)
# print(predictions)

1/1 ━━━━━━━━━━━━━━━━━━━━ 5s 5s/step


In [17]:
train_loss, train_accuracy = model.evaluate(X_train, y_train)
print(f"Training data Loss: {train_loss}")
print(f"Training data Accuracy: {train_accuracy}")

2/2 ━━━━━━━━━━━━━━━━━━━━ 5s 30ms/step - accuracy: 0.2951 - loss: 2.5190
Training data Loss: 2.5839309692382812
Training data Accuracy: 0.2708333432674408


In [18]:
val_loss, val_accuracy = model.evaluate(X_val, y_val)
print(f"Validation Loss: {val_loss}")
print(f"Validation Accuracy: {val_accuracy}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 129ms/step - accuracy: 0.2000 - loss: 3.7738
Validation Loss: 3.773798704147339
Validation Accuracy: 0.20000000298023224


In [19]:
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 120ms/step - accuracy: 0.3636 - loss: 3.1048
Test Loss: 3.104775905609131
Test Accuracy: 0.3636363744735718


In [24]:
count = 0
for arr in y_train:
    if arr[0] == 1:
        count+=1
print("The MVP is the highest scorer ", count/len(y_train), " of the time in the training set")

The MVP is the highest scorer  0.2708333333333333  of the time in the training set


In [25]:
count = 0
for arr in y_val:
    if arr[0] == 1:
        count+=1
print("The MVP is the highest scorer ", count/len(y_val), " of the time in the validation set")

The MVP is the highest scorer  0.2  of the time in the validation set


In [26]:
count = 0
for arr in y_test:
    if arr[0] == 1:
        count+=1
print("The MVP is the highest scorer ", count/len(y_val), " of the time in the test set")

The MVP is the highest scorer  0.4  of the time in the test set


In [28]:
predict_yr = "2025"

data = get_final_df_by_year(predict_yr).to_numpy()[:, :-1]
shaped_data = data.reshape(1, 75, 31)
prediction_arr = model.predict(shaped_data)
mvp_idx = prediction_arr[0].argmax()
top_10_indices = prediction_arr[0].argsort()[-10:][::-1]

mvp = get_top_75_filtered(get_data(predict_yr))['Player'][mvp_idx]
top_10_mvps = get_top_75_filtered(get_data(predict_yr))['Player'][top_10_indices]
print(mvp)
print(top_10_mvps)

found  0  MVPs in year 2025
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step
Giannis Antetokounmpo
0     Giannis Antetokounmpo
1             Anthony Davis
2               LaMelo Ball
5            Paolo Banchero
3              Nikola Jokić
6              De'Aaron Fox
4              Jayson Tatum
18            Jalen Brunson
20           Brandon Ingram
9           Anthony Edwards
Name: Player, dtype: object
